# Analyzing Stock Sentiment from Twits


### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import pandas as pd
import numpy as np
import pickle 
from tqdm import tqdm

from torch import nn, optim
import torch.nn.functional as F

# Load pre-trained Embedding

In [2]:
words = []
idx = 0
word2idx = {}
vectors = []
#vectors = bcolz.carray(np.zeros(1), rootdir='glove.6B.100d.txt', mode='w')

with open('glove.6B.100d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
        
glove = {w: vectors[word2idx[w]] for w in words}

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [3]:
with open(os.path.join('twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [5]:
twits['data'][:2]

[{'message_body': '$FITB great buy at 26.00...ill wait',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:00:09Z'},
 {'message_body': '@StockTwits $MSFT',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:00:42Z'}]

In [6]:
"""print out the number of twits"""

# TODO Implement 

len(twits['data'])

1548010

### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [2]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'https?://[^\s]+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[a-zA-Z0-9]*', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@[a-zA-Z0-9]*', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z]', ' ', text)

    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w, 'v') for w in tokens if len(w)  > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
tokenized = []
for message in tqdm(messages):
    tokenized.append(preprocess(message))

100%|██████████| 1548010/1548010 [02:52<00:00, 8994.47it/s] 


In [7]:
with open('tokenized_ori.pickle', 'wb') as f:
    pickle.dump(tokenized, f)

In [8]:
tokenized[:3]

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'of',
  'hold',
  'set',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'be',
  'buy']]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
non_empty_tokens = [idx for idx, token in enumerate(tokenized) if len(token)>0]
tokenized = [tokenized[idx] for idx in non_empty_tokens]
sentiments = [sentiments[idx] for idx in non_empty_tokens]

In [10]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""

words = [word for twit in tokenized for word in twit]
bow = Counter(words)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [11]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {key: val/len(words) for key, val in bow.items()}
ff = [val/len(words) for key, val in bow.items()]
# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 2e-7

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 5

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['be', 'the', 'to', 'for', 'on']


28956

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [12]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: i+1 for i, word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {i: word for word, i in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = []
for token in tqdm(tokenized):
    filtered.append([word for word in token if word in filtered_words])

100%|██████████| 1499486/1499486 [36:48<00:00, 678.81it/s] 


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [13]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in tqdm(enumerate(sentiments)):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

1499486it [00:02, 556199.92it/s]


If you did it correctly, you should see the following result 

In [14]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19940619970705337

Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

### Insert pre-trained embedding

In [32]:
matrix_len = len(filtered_words)
weights_matrix = np.zeros((matrix_len+1, 100))
words_found = 0
for i, word in enumerate(filtered_words):
    try:
        weights_matrix[i+1] = glove[word]
        words_found += 1
    except:
        weights_matrix[i+1] = np.random.normal(scale=0.6, size=(100, ))
weights_matrix[0] = np.random.normal(scale=0.6, size=(100, ))
print('match ratio is ', words_found/matrix_len)

match ratio is  0.793859649122807


In [8]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embedding, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embedding, embedding_dim)
    emb_layer.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    if non_trainable:
        emb_layer.weight.requires_grad = False
    
    return emb_layer, num_embedding, embedding_dim

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [3]:
train_on_gpu = torch.cuda.is_available()
train_on_gpu

True

In [9]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, 
                 lstm_layers=1, dropout=0.2, bidirectional=True, cov1d_ch=4, kernel_size=3 ):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        

        # Setup embedding layer
        # For non-pre-trained embedding layer
        #self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embed_size)
        # For pre-trained embedding layer
        self.embedding, num_embeddings, self.embed_size = create_emb_layer(weights_matrix, False)
        
        # Setup additional layers
        
        self.dropout = nn.Dropout(p=dropout)
      
        self.gru = nn.GRU(input_size=self.embed_size, hidden_size=self.lstm_size,
                          num_layers=self.lstm_layers, batch_first=False, bidirectional=self.bidirectional)
        
        if self.bidirectional:
            self.fc = nn.Linear(self.lstm_size*2, self.output_size)
            self.bn = nn.BatchNorm1d(self.lstm_size*2)
        else:
            self.fc = nn.Linear(  self.lstm_size, self.output_size)
            self.bn = nn.BatchNorm1d( self.lstm_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # Createnew tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        if train_on_gpu:
            if self.bidirectional:
                hidden = weight.new(self.lstm_layers*2, batch_size, self.lstm_size).zero_().cuda()
            else:
                hidden = weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda()
        else:
            if self.bidirectional:
                hidden = weight.new(self.lstm_layers*2, batch_size, self.lstm_size).zero_()
            else:
                hidden = weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_()
        
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        embeds = self.embedding(nn_input)
        
        gru_out, hidden = self.gru(embeds, hidden_state)
        if self.bidirectional:
            out = gru_out[-1].contiguous().view(-1, self.lstm_size*2)
        else:
            gru_out = gru_out[-1]
            out = gru_out.contiguous().view(-1, self.lstm_size)
            
        out = self.fc(out)
        out = self.log_softmax(out)
        
        return out, hidden

### Test Model

In [10]:
inputs = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
seq_length, batch_size = inputs.size()
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=1, bidirectional=False)
model.embedding.weight.data.uniform_(-1, 1)

hidden = model.init_hidden(4).data
if train_on_gpu:
    model = model.cuda()
    hidden = hidden.cuda()
    inputs = inputs.cuda()

logps, _ = model(inputs, hidden)
print(logps)

tensor([[-2.3316, -1.5958, -1.5843, -1.3258, -1.4721],
        [-2.3609, -1.6422, -1.6177, -1.4544, -1.2722],
        [-1.7790, -1.5578, -1.4155, -1.6782, -1.6551],
        [-2.5797, -1.5023, -1.8507, -1.6051, -1.0683]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [22]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

split_size = 0.9

split_idx = int(len(token_ids)*split_size)
train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]


In [ ]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
            
        labels = np.array(labels)
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        yield batch, label_tensor

# Load/Save preprocessed data

In [6]:
# Load
with open('tokenized.pickle', 'rb') as f:
    tokenized = pickle.load(f)
with open('filtered_words.pickle', 'rb') as f:
    filtered_words = pickle.load(f)
with open('vocab.pickle', 'rb') as f:
    vocab = pickle.load(f)
with open('train_features.pickle', 'rb') as f:
    train_features = pickle.load(f)
with open('valid_features.pickle', 'rb') as f:
    valid_features = pickle.load(f)
with open('train_labels.pickle', 'rb') as f:
    train_labels = pickle.load(f)
with open('valid_labels.pickle', 'rb') as f:
    valid_labels = pickle.load(f)
with open('weights_matrix', 'rb') as f:
    weights_matrix = pickle.load(f)

# Save
"""
with open('weights_matrix', 'wb') as f:
    pickle.dump(weights_matrix, f)
with open('filtered_words.pickle', 'wb') as f:
    pickle.dump(filtered_words, f)
with open('vocab.pickle', 'wb') as f:
    pickle.dump(vocab, f)
with open('train_features.pickle', 'wb') as f:
    pickle.dump(train_features, f)
with open('valid_features.pickle', 'wb') as f:
    pickle.dump(valid_features, f)
with open('train_labels.pickle', 'wb') as f:
    pickle.dump(train_labels, f)
with open('valid_labels.pickle', 'wb') as f:
    pickle.dump(valid_labels, f)
"""

Showing example

In [26]:
sequence_length = 15
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=sequence_length, batch_size=64)))
batch_size = text_batch.size()[-1]
model = TextClassifier(len(vocab)+1, 200, 64, 5, dropout=0., bidirectional=False)
hidden = model.init_hidden(64)
if train_on_gpu:
    model = model.cuda()
    hidden = hidden.cuda()
    text_batch = text_batch.cuda()
logps, hidden = model(text_batch, hidden)


### Training
It's time to train the neural network!

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(vocab)+1
embed_size = 500
lstm_size = 256
output_size = 5
lstm_layers = 2
dropout = 0.2
bidirectional = False
batch_size = 256
sequence_length = 30

model = TextClassifier(vocab_size, embed_size, lstm_size, output_size,
                       lstm_layers=lstm_layers, dropout=dropout, bidirectional=bidirectional)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(28957, 100)
  (dropout): Dropout(p=0.2)
  (gru): GRU(100, 256, num_layers=2)
  (fc): Linear(in_features=256, out_features=5, bias=True)
  (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (log_softmax): LogSoftmax()
)

In [32]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 2
learning_rate = 0.001
clip = 3


print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, amsgrad=True, betas=(0.9,0.999))

val_loss_min = np.inf

model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    hidden = model.init_hidden(batch_size)
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=True):
        if text_batch.size() != torch.Size([sequence_length, batch_size]):
            continue
        steps +=1
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        hidden = hidden.data.to(device)

        model.zero_grad()

        output, hidden = model(text_batch, hidden)
        loss = criterion(output, labels)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()


        if steps % print_every == 0:
            model.eval()
            val_h = model.init_hidden(batch_size)
            val_losses = []
            val_accuracy = []

            for inputs, val_labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length=sequence_length):
                if inputs.size() != torch.Size([sequence_length, batch_size]):
                    continue
                val_h = val_h.data.to(device)
        
                inputs, val_labels = inputs.to(device), val_labels.to(device)
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output, val_labels)
                if val_loss.item() < val_loss_min:
                    val_loss_min = val_loss
                    #print('save model...')
                    torch.save(model, 'model.pt')
                val_losses.append(val_loss.item())
                
                val_ps = torch.exp(output)
                top_p, top_class = val_ps.topk(1, dim=1)
                equals = top_class == val_labels.view(*top_class.shape)
                val_accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())

            model.train()
            this_val_acc = sum(val_accuracy)/len(val_accuracy)
            
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                      "Step: {}...".format(steps),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}...".format(np.mean(val_losses)),
                     "Val Accuracy: {:6f}".format(this_val_acc))

Starting epoch 1


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type TextClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1/2... Step: 100... Loss: 1.275868... Val Loss: 1.278801... Val Accuracy: 0.474701
Epoch: 1/2... Step: 200... Loss: 1.063151... Val Loss: 1.153116... Val Accuracy: 0.538108
Epoch: 1/2... Step: 300... Loss: 1.073083... Val Loss: 1.081537... Val Accuracy: 0.579051
Epoch: 1/2... Step: 400... Loss: 0.881566... Val Loss: 1.005686... Val Accuracy: 0.606539
Epoch: 1/2... Step: 500... Loss: 0.853084... Val Loss: 0.965131... Val Accuracy: 0.626302
Epoch: 1/2... Step: 600... Loss: 0.910250... Val Loss: 0.950454... Val Accuracy: 0.626871
Epoch: 1/2... Step: 700... Loss: 0.833750... Val Loss: 0.913231... Val Accuracy: 0.646943
Epoch: 1/2... Step: 800... Loss: 0.823990... Val Loss: 0.890652... Val Accuracy: 0.662269
Epoch: 1/2... Step: 900... Loss: 0.824825... Val Loss: 0.862991... Val Accuracy: 0.672251
Epoch: 1/2... Step: 1000... Loss: 0.845440... Val Loss: 0.851416... Val Accuracy: 0.681684
Epoch: 1/2... Step: 1100... Loss: 0.830605... Val Loss: 0.841420... Val Accuracy: 0.686073
Epoch: 1

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [16]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """ 
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1).to('cpu')
    # Get the NN output
    hidden = model.init_hidden(text_input.size(1)).to('cpu')
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred.detach().numpy()

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

In [ ]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
pred = predict(text, model, vocab)

In [18]:
print("Class {} is the most probable class with a likelihood of {:.2f}%".format(np.argmax(pred)-2, np.max(pred)*100))

Class 1 is the most probable class with a likelihood of 76.66%


Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [9]:
with open(os.path.join('test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [20]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [21]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        tokens = preprocess(text)
        tokens = [word for word in tokens if word in filtered_words]
        if len(tokens) > 0:
            #print(text)
            score = predict(text, model, vocab)

            for symbol in symbols:
                if symbol in universe:
                    
                    yield {'text': text, 'symbol': symbol, 'score': score, 'timestamp': twit['timestamp'], 'sentiment': np.arange(0,5)[np.argmax(score)]}

In [22]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'text': '$AAPL 190 after ER',
 'symbol': '$AAPL',
 'score': array([[ 0.15485731,  0.16452152,  0.18032193,  0.14882466,  0.35147461]], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z',
 'sentiment': 4}

In [23]:
next(score_stream)

{'text': '$AMZN A lot of technical signs point to a short-term bottom in place for stocks (John Murphy)',
 'symbol': '$AMZN',
 'score': array([[  3.03409062e-04,   3.74427512e-02,   3.36010493e-02,
           9.09555435e-01,   1.90973319e-02]], dtype=float32),
 'timestamp': '2018-11-01T00:00:42Z',
 'sentiment': 3}

In [24]:
next(score_stream)

{'text': 'Long Only: $BAC, $CGC, and $EXTR.\nLong &amp; Puts: $IQ.\nShort: $MU, $AMD, and $NVDA.',
 'symbol': '$MU',
 'score': array([[ 0.02309324,  0.16099609,  0.05883843,  0.15040793,  0.6066643 ]], dtype=float32),
 'timestamp': '2018-11-01T00:02:10Z',
 'sentiment': 4}

In [25]:
next(score_stream)

{'text': '$AMZN Like what i see here',
 'symbol': '$AMZN',
 'score': array([[ 0.06128268,  0.12226567,  0.44065145,  0.29710418,  0.07869604]], dtype=float32),
 'timestamp': '2018-11-01T00:03:35Z',
 'sentiment': 2}

In [26]:
next(score_stream)

{'text': '@TraderLeibniz $NVDA Perhaps, no chance for $MU and $AMD till cycle &amp; trade war no longer concerns.... played these names for years. GL',
 'symbol': '$MU',
 'score': array([[ 0.2873998 ,  0.2912277 ,  0.11499511,  0.26970211,  0.03667523]], dtype=float32),
 'timestamp': '2018-11-01T00:04:21Z',
 'sentiment': 1}

That's it. You have successfully built a model for sentiment analysis! 